In [1]:
import os
import numpy as np
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
        gpus[int(0)],
        [tf.config.LogicalDeviceConfiguration(memory_limit=15000)])
        tf.config.experimental.set_visible_devices(gpus[int(0)], 'GPU')
        # tf.config.experimental.set_memory_growth(gpus[int(SELECTED_GPU)], True)
        # tf.config.experimental.se.set_per_process_memory_fraction(0.92)
        print(gpus[int(0)])
    except RuntimeError as e:
        print(e)
else:
    print('GPU not found!')

# import matplotlib.pyplot as plt

from scipy.stats import pearsonr, spearmanr

import datasets
from tqdm.auto import tqdm

from transformers import (
    AutoTokenizer,
    TFBertForSequenceClassification
)

2022-11-25 21:02:50.012977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 21:02:50.144960: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-25 21:02:50.763830: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/modaresi/.conda/envs/globenc-venv/lib/
2022-11-25 21:02:50.763939: W tensorflow/

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2022-11-25 21:02:51.692057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 21:02:51.699162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-25 21:02:51.699311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
MODELS = {
    "mnli-e0": "bert-base-uncased",
    "mnli-e1": "/home/modaresi/projects/globenc_analysis/outputs/models/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-12272",
    "mnli-e2": "/home/modaresi/projects/globenc_analysis/outputs/models/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-24544",
    "mnli-e3": "/home/modaresi/projects/globenc_analysis/outputs/models/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-36816",
    "mnli-e4": "/home/modaresi/projects/globenc_analysis/outputs/models/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-49088",
    "mnli-e5": "/home/modaresi/projects/globenc_analysis/outputs/models/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360",
}

TASK = "mnli"
 
SET = "train"  # train/validation/validation_matched

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

MAX_LENGTH = 128

actual_task = "mnli" if TASK == "mnli-mm" else TASK
dataset = datasets.load_dataset("glue", actual_task)
# metric = datasets.load_metric('glue', actual_task)
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
SENTENCE1_KEY, SENTENCE2_KEY = task_to_keys[TASK]
tokenizer = None
sel_dataset = None
dataset

Reusing dataset glue (/opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [4]:
def preprocess_function_wrapped(tokenizer):
    def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[SENTENCE1_KEY],) if SENTENCE2_KEY is None else (examples[SENTENCE1_KEY], examples[SENTENCE2_KEY])
        )
        result = tokenizer(*args, padding=False, max_length=MAX_LENGTH, truncation=True)
        return result
    return preprocess_function

def token_id_to_tokens_mapper(tokenizer, sample):
    length = len(sample["input_ids"])
    return tokenizer.convert_ids_to_tokens(sample["input_ids"])[:length], length

from IPython.display import display, HTML
# def print_globenc(globenc, tokenized_text, discrete=False, prefix=""): 
#     if len(globenc.shape) == 2: 
#         globenc = np.expand_dims(globenc, axis=0) 
#     # norm_cls = globenc
#     # norm_cls = np.flip(norm_cls, axis=0) 
#     # row_sums = norm_cls.max(axis=1) 
#     # norm_cls = norm_cls / row_sums[:, np.newaxis] 
#     html = prefix 
#     if discrete: 
#         cls_attention = np.argsort(np.argsort(norm_cls[0, :])) / len(norm_cls[0, :]) 
#     else: 
#         cls_attention = globenc / globenc.max()
#     for i in range(len(tokenized_text)): 
#         html += (f"<span style='background-color: rgba(10, {cls_attention[i]*255}, 10, {cls_attention[i] / 1.5}); " 
#                  f"font-size: {int(cls_attention[i]*18 + 1)}px; " 
#                  f"font-weight: {int(cls_attention[i]*900)};'>") 
#         html += tokenized_text[i] 
#         html += "</span> " 
#     display(HTML(html))

def print_globenc(globenc, tokenized_text, discrete=False, prefix="", no_cls=False):
    # if len(globenc.shape) == 2:
    #     globenc = np.expand_dims(globenc, axis=0)
    # norm_cls = globenc[:, 0, :]
    # norm_cls = np.flip(norm_cls, axis=0)
    # row_sums = norm_cls.max(axis=1)
    # norm_cls = norm_cls / row_sums[:, np.newaxis]
    html = prefix
    if discrete:
        cls_attention = np.argsort(np.argsort(norm_cls[0, :])) / len(norm_cls[0, :])
    else:
        cls_attention = globenc / globenc.max()
    for i in range(len(tokenized_text)):
        html += (f"<span style='"
                 f"background-color: rgba({cls_attention[i]*255}, {cls_attention[i]*255}, 0, {cls_attention[i] / 1.5}); "
#                  f"background-color: rgba(200, {cls_attention[i]*255}, 10, 1.0); "
#                  f"font-size: {int(cls_attention[i]*18 + 1)}px; "
#                  f"font-weight: {int(cls_attention[i]*900)};"
                 f"font-weight: {int(800)};"
                 "'>")
        html += tokenized_text[i]
        html += "</span> "
    display(HTML(html))

In [12]:
idx = 192454
HTAs = []
HTAs_angular = []
for name, path in tqdm(MODELS.items(), desc="Models"):
    model = TFBertForSequenceClassification.from_pretrained(path, from_pt=True, output_hidden_states=True)
    tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True, max_length=128) if tokenizer is None else tokenizer
    sel_dataset = dataset[SET].map(preprocess_function_wrapped(tokenizer), batched=True, batch_size=1024) if sel_dataset is None else sel_dataset
    dataset_size = len(sel_dataset)

    tokens, length = token_id_to_tokens_mapper(tokenizer, sel_dataset[idx])
    @tf.function(input_signature=[{'input_ids': tf.TensorSpec(shape=[1, None], dtype=tf.int32),
             'token_type_ids': tf.TensorSpec(shape=[1, None], dtype=tf.int32),
             'attention_mask': tf.TensorSpec(shape=[1, None], dtype=tf.int32)}, 
             tf.TensorSpec(shape=(1,), dtype=tf.int32)])
    def get_grads(inputs, i):
        with tf.GradientTape() as tape:
            outputs = model(inputs)
            input_embeds = outputs.hidden_states[0]
            out_embeds = tf.gather(outputs.hidden_states[-1], i, batch_dims=1)

        grads = tape.batch_jacobian(out_embeds, input_embeds)
        print(grads)
        return grads * tf.expand_dims(input_embeds, axis=1)
    
    hta = np.zeros((length, length), dtype=np.float32)
    hta_angular = np.zeros((length, length), dtype=np.float32)

    for i in tqdm(range(length)):
        feats = {
            'input_ids': tf.constant([sel_dataset[idx]['input_ids']], dtype=tf.int32),
            'attention_mask': tf.constant([sel_dataset[idx]['attention_mask']], dtype=tf.int32),
            'token_type_ids': tf.constant([sel_dataset[idx]['token_type_ids']], dtype=tf.int32),
        }
        grads = get_grads(feats, tf.constant([i])) 
        hta[i, :length] = np.linalg.norm(grads, axis=(1,3))
        hta_angular[i, :length] = np.sum(grads, axis=(1,3))
    HTAs.append(hta)
    HTAs_angular.append(hta_angular)
    # if prev_HTA is not None:
    #     print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    # prev_HTA = hta
    # print_globenc(hta[0], tokens, prefix=name + " ")

Models:   0%|          | 0/6 [00:00<?, ?it/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/29 [00:00<?, ?it/s]

Tensor("Reshape_2:0", shape=(1, 768, None, 768), dtype=float32)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


...

  0%|          | 0/29 [00:00<?, ?it/s]

Tensor("Reshape_2:0", shape=(1, 768, None, 768), dtype=float32)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


  0%|          | 0/29 [00:00<?, ?it/s]

Tensor("Reshape_2:0", shape=(1, 768, None, 768), dtype=float32)


In [20]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 132919 ###


Corr: 0.3138103142287821


Corr: 0.9636677276493726


Corr: 0.979306589060537


Corr: 0.9261533454221914


Corr: 0.9722983159460893


In [18]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 61630 ###


Corr: 0.2706520938194725


Corr: 0.9808144908140684


Corr: 0.9911796638662177


Corr: 0.8457896193430214


Corr: 0.9826494119924254


In [16]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 223115 ###


Corr: 0.7521692099234305


Corr: 0.971653861606745


Corr: 0.8519427756798851


Corr: 0.8611374043680651


Corr: 0.9782125558850623


In [14]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 113001 ###


Corr: 0.6653317691578715


Corr: 0.9222954141695432


Corr: 0.9633811836065308


Corr: 0.8521806895455177


Corr: 0.9635059556718347


In [10]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 364274 ###


Corr: 0.4760787503330629


Corr: 0.9329211157773295


Corr: 0.7850461788517407


Corr: 0.9809180133618233


Corr: 0.9974738718404607


In [6]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 81139 ###


Corr: 0.5240994060964408


Corr: 0.6566756440413792


Corr: 0.9580572241879687


Corr: 0.6978351736216577


Corr: 0.9967892772357397


In [8]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 314093 ###


Corr: 0.7074305929199067


Corr: 0.9521136091294028


Corr: 0.9896806422009397


Corr: 0.9790491184104891


Corr: 0.9993519332837945


In [12]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 24638 ###


Corr: 0.5136071945680656


Corr: 0.9703191507966203


Corr: 0.9483345963657646


Corr: 0.9794906825233944


Corr: 0.9981393079170179


In [10]:
hta[0] - hta[0].min()

array([2.3919744 , 1.6143422 , 1.4383541 , 1.6806723 , 1.6584038 ,
       2.1515262 , 2.461934  , 2.2538314 , 1.6900308 , 1.3193939 ,
       2.0133724 , 2.0184886 , 1.7156214 , 2.6452494 , 1.548722  ,
       1.7915325 , 1.6018206 , 1.5491389 , 2.184195  , 0.11396325,
       1.9367687 , 1.6886852 , 1.8591691 , 2.245592  , 2.0483835 ,
       1.6859921 , 0.        , 2.0205436 , 1.1432397 ], dtype=float32)

In [23]:
hta[0]

array([0.85667294, 0.5133918 , 0.5688279 , 0.5069518 , 0.36562893,
       0.5855837 , 0.77655613, 0.5806771 , 3.1768572 , 0.98984784,
       0.54174685, 0.5073529 , 0.53512627, 0.6214964 , 0.42636976,
       0.62936515, 0.64160085, 3.2990708 , 0.8523279 , 1.3776093 ,
       0.6884577 , 0.416046  , 0.36505693, 0.55093414, 0.68505704,
       0.3553516 , 2.4756973 , 0.8500618 , 0.34904358], dtype=float32)

In [27]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    mix_hta = hta[0] * HTAs_angular[e_no][0]
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA, mix_hta)[0])
    prev_HTA = mix_hta
    print_globenc(mix_hta - mix_hta.min(), tokens, prefix=f"Epoch {e_no}")

### 192454 ###


Corr: 0.08134321569262973


Corr: 0.41049485720212947


Corr: 0.4518017572843218


Corr: -0.7352775066821189


Corr: 0.8611231998650992


In [11]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0] - hta[0].min(), tokens, prefix=f"Epoch {e_no}")

### 192454 ###


Corr: 0.14754363123060987


Corr: 0.5902791736872869


Corr: 0.500454555960453


Corr: -0.44109499859960094


Corr: 0.8838220579893918


In [18]:
prev_HTA = None
print(f"### {idx} ###")
for e_no, hta in enumerate(HTAs):
    if prev_HTA is not None:
        print("Corr:", pearsonr(prev_HTA[0], hta[0])[0])
    prev_HTA = hta
    print_globenc(hta[0], tokens, prefix=f"Epoch {e_no}")

### 192454 ###


Corr: 0.3436310163785424


Corr: 0.8465793280696551


Corr: 0.965153608197307


Corr: 0.9861460380876459


Corr: 0.9980208082363609
